In [10]:
from bs4 import BeautifulSoup

import re
import requests

In [11]:
BASE = 'https://stackoverflow.com'
TAG = 'machine-learning'
PAGES = 5

In [12]:
def get_hrefs(page: int) -> list[str]:
    res = requests.get(f'{BASE}/questions/tagged/{TAG}?pagesize=50&page={page}&tab=Frequent')
    soup = BeautifulSoup(res.content, 'lxml')
    return [i.get('href') for i in soup.select('a[class=s-link]')]

In [13]:
def get_content(href: str) -> str:
    res = requests.get(f'{BASE}{href}')
    soup = BeautifulSoup(res.content, 'lxml')
    posts = soup.select('div.s-prose.js-post-body')

    content = []
    for post in posts:
        content += post.find_all('p')
    
    return ''.join(f'{i.text} ' for i in content).strip()

In [14]:
with open(f'data/{TAG}.txt', 'w', encoding='utf-8') as fp:
    for p in range(1, PAGES+1):
        hrefs = get_hrefs(p)
        for i, h in enumerate(hrefs):
            print(f'Page #{p}, Post #{i} ', end='\r')
            fp.write(get_content(h))

In [ ]:
with open(f'data/{TAG}.txt', 'r', encoding='utf-8') as fp:
    text = fp.read()

site_pattern = r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)'
re.sub(site_pattern, ' ', text[:1500])